# Tarea 5
## Clustering
### María José Liberona

## Importación librerías


In [ ]:
#Librerías
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## Subir y leer archivo


In [ ]:
#Subir archivo 
from google.colab import files
uploaded = files.upload()  # Frogs_MFCCs.csv

Saving Frogs_MFCCs.csv to Frogs_MFCCs.csv


In [ ]:
#Leer archivo csv
data=pd.read_csv('Frogs_MFCCs.csv')

#Divide la data en características (MFCCs_1-MFCCs_22) y labels (Species)
MFCCs_data=data.iloc[:,:22]
labels_data=data.iloc[:,24]

#Reemplazo de las especies por números de 0 a 9
labels_data.loc[labels_data=='AdenomeraAndre']=0
labels_data.loc[labels_data=='AdenomeraHylaedactylus']=1
labels_data.loc[labels_data=='Ameeregatrivittata']=2
labels_data.loc[labels_data=='HylaMinuta']=3
labels_data.loc[labels_data=='HypsiboasCinerascens']=4
labels_data.loc[labels_data=='HypsiboasCordobae']=5
labels_data.loc[labels_data=='LeptodactylusFuscus']=6
labels_data.loc[labels_data=='OsteocephalusOophagus']=7
labels_data.loc[labels_data=='Rhinellagranulosa']=8
labels_data.loc[labels_data=='ScinaxRuber']=9

labels_data=labels_data.to_numpy().astype(int)
#Muestra los datos
print(MFCCs_data)
print(labels_data)

#Columnas
columnas_data=["MFCCs_" + str(i) for i in range(1,23)] #Nombre de las características
print(columnas_data)

      MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  ...  MFCCs_20  MFCCs_21  MFCCs_22
0          1.0  0.152936 -0.105586  ...  0.057684  0.118680  0.014038
1          1.0  0.171534 -0.098975  ...  0.020140  0.082263  0.029056
2          1.0  0.152317 -0.082973  ... -0.025083  0.099108  0.077162
3          1.0  0.224392  0.118985  ... -0.054766 -0.018691  0.023954
4          1.0  0.087817 -0.068345  ... -0.031346  0.108610  0.079244
...        ...       ...       ...  ...       ...       ...       ...
7190       1.0 -0.554504 -0.337717  ...  0.052449 -0.021860 -0.079860
7191       1.0 -0.517273 -0.370574  ...  0.046461 -0.015418 -0.101892
7192       1.0 -0.582557 -0.343237  ...  0.027834 -0.000531 -0.080425
7193       1.0 -0.519497 -0.307553  ...  0.041803 -0.027911 -0.096895
7194       1.0 -0.508833 -0.324106  ...  0.031560 -0.029355 -0.087910

[7195 rows x 22 columns]
[0 0 0 ... 9 9 9]
['MFCCs_1', 'MFCCs_2', 'MFCCs_3', 'MFCCs_4', 'MFCCs_5', 'MFCCs_6', 'MFCCs_7', 'MFCCs_8', 'MFCCs_9', 'MFCCs_10', 'MFC

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


## Define our evaluation benchmark

We will first our evaluation benchmark. During this benchmark, we intend to
compare different initialization methods for KMeans. Our benchmark will:

* create a pipeline which will scale the data using a
  :class:`~sklearn.preprocessing.StandardScaler`;
* train and time the pipeline fitting;
* measure the performance of the clustering obtained via different metrics.



In [ ]:
def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time() #Tiempo de inicio 
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data) #Estima y normaliza los datos
    fit_time = time() - t0 #Calcula el tiempo de entrenamiento
    results = [name, fit_time, estimator[-1].inertia_] #resultados
    n_clusters= len(set((estimator[-1].labels_))) # número de clusters

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics] #Calcula el valor de los clusters a partir del clustering Kmeans
    #print(results)
    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean", sample_size=300,)
    ]

    results += [n_clusters]
    # Show the results
    formatter_result = ("{:15s}\t\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:.0f}")
    print(formatter_result.format(*results))

## Run the benchmark

We will compare three approaches:

* an initialization using `kmeans++`. This method is stochastic and we will
  run the initialization 4 times;
* a random initialization. This method is stochastic as well and we will run
  the initialization 4 times;
* an initialization based on a :class:`~sklearn.decomposition.PCA`
  projection. Indeed, we will use the components of the
  :class:`~sklearn.decomposition.PCA` to initialize KMeans. This method is
  deterministic and a single initialization suffice.



In [ ]:
#Ejemplo
print(105 * '_')
print('init\t\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\tclusters')

kmeans = KMeans(init="k-means++", n_clusters=10, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=MFCCs_data, labels=labels_data)

kmeans = KMeans(init="random", n_clusters=10, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=MFCCs_data, labels=labels_data)

pca = PCA(n_components=10).fit(MFCCs_data)
kmeans = KMeans(init=pca.components_, n_clusters=10, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=MFCCs_data, labels=labels_data)

print(105 * '_')

_________________________________________________________________________________________________________
init			time	inertia	homo	compl	v-meas	ARI	AMI	silhouette	clusters
k-means++      		0.274s	51996	0.708	0.571	0.633	0.432	0.631	0.225		10
random         		0.306s	54400	0.641	0.507	0.566	0.326	0.565	0.195		10
PCA-based      		0.080s	58266	0.720	0.698	0.709	0.863	0.708	0.387		10
_________________________________________________________________________________________________________


## Función bench_clustering2()

Basándose en la función bench_k_means() considerando las siguientes indicaciones:

a) DBSCAN no posee la variable inertia._
b) DBSCAN no asigna todas las muestras a un cluster, las muestras no asociadas tienen label '-1', por ello debe entregar la opción de: (i) Calcular las métricas sólo con los datos asociados o (ii) calcular las métricas usando todos los datos, asumiendo que los datos no asociados conforman un cluster extra. Dicha opción debe ser implementada usando una variable extra (True/False) que se le entrega a la función.
 

In [ ]:
def bench_clustering2(fc, name, data, labels, all): #Función que sirve para el DBSCAN y el método aglomerativo
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    db or agg : DBSCAN instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """

    t0 = time() #Tiempo de inicio 
    data_n=StandardScaler().fit_transform(data) #Normaliza los datos
    estimator = fc.fit(data_n) #Entrena el estimador a partir de los datos normalizados
    fit_time = time() - t0 #Tiempo de entrenamiento
    labels_pred=np.array(estimator.labels_).astype(int) #Predice el labels de cada muestra
    largo=len(labels_pred) 
    #print(labels_pred)
    results = [name, fit_time, np.nan]
    try:
      #Antes de calcular las métricas se debe considerar si se requiere utilizar todos los datos o no
      if all==True: #Considera todas las muestras
        labels_pred[labels_pred==-1]=max(set(labels_pred))+1 #Cambia los -1 al n° mayor de cluster+1
        #Se redefinen los arreglos
        data_n1=data_n
        labels_data1=labels_data
        labels_pred1=labels_pred
        #print(set(labels_pred))
        n_clusters= len(set(labels_pred1)) #Calcula el total de clusters

      elif all==False: #No considera todas las muestras
        data_n1=pd.DataFrame([]) 
        labels_data1=np.array([])
        labels_pred1=np.array([])
        for i in range(largo):
          if labels_pred[i]!=-1: #Si tiene asignado un cluster entonces se agrega el dato 
            data_n1=data_n1.append(pd.Series(data_n[i,:]), ignore_index=True)
            labels_data1=np.append(labels_data1,labels_data[i])
            labels_pred1=np.append(labels_pred1, labels_pred[i])
        n_clusters= len(set(labels_pred1))
      # Define the metrics which require only the true labels and estimator
      # labels

      clustering_metrics = [
          metrics.homogeneity_score,
          metrics.completeness_score,
          metrics.v_measure_score,
          metrics.adjusted_rand_score,
          metrics.adjusted_mutual_info_score
      ]
      results += [m(labels_data1, labels_pred1) for m in clustering_metrics] #Calcula las métricas a partir de las predicciones y labels originales

      # The silhouette score requires the full dataset
      if n_clusters==0 or n_clusters==1: #Si n_clusters da 0 o 1
        results +=[np.nan]

      else:
        results += [
            metrics.silhouette_score(data_n1, labels_pred1,
                                  metric="euclidean", sample_size=300,)
        ]
      
      #Añade n_clusters en la tabla
      results += [n_clusters]
      # Show the results
      formatter_result = ("{:15s}\t\t{:.3f}s\t{:.3f}\t{:.3f}\t{:.3f}"
                          "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:.0f}")
      print(formatter_result.format(*results))
    
    except:
      print(str(name)+'          No se pueden calcular las métricas')

In [ ]:
#Ejemplo
print(105 * '_')
print('init\t\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\tclusters')

db = DBSCAN(eps=0.5)
bench_clustering2(fc=db, name="DBSCAN", data=MFCCs_data, labels=labels_data, all=True)

Agg = AgglomerativeClustering(n_clusters=10)
bench_clustering2(fc=Agg, name="Aglomerativo", data=MFCCs_data, labels=labels_data, all=True)

print(105 * '_')

_________________________________________________________________________________________________________
init			time	inertia	homo	compl	v-meas	ARI	AMI	silhouette	clusters
DBSCAN         		0.579s	nan	0.074	0.451	0.127	0.049	0.123	-0.212		9
Aglomerativo   		2.205s	nan	0.789	0.661	0.719	0.590	0.718	0.308		10
_________________________________________________________________________________________________________


## Pruebas con distintas variantes de algoritmos




In [ ]:
#Pruebas con distintas variantes de algoritmos

print(105 * '_')
print('init\t\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\tclusters')
#K-Means con inicialización al azar
kmeans = KMeans(init="random", n_clusters=10)
bench_k_means(kmeans=kmeans, name="random", data=MFCCs_data, labels=labels_data)

#K-means++
kmeans = KMeans(init="k-means++", n_clusters=10)
bench_k_means(kmeans=kmeans, name="k-means++", data=MFCCs_data, labels=labels_data)

#DBSCAN con épsilon por defecto
db = DBSCAN() 
bench_clustering2(fc=db, name="DBSCAN-f 0.5", data=MFCCs_data, labels=labels_data, all=False)

#DBSCAN con épsilon 0.7
db = DBSCAN(eps=0.7) 
bench_clustering2(fc=db, name="DBSCAN-f 0.7", data=MFCCs_data, labels=labels_data, all=False)

#DBSCAN con épsilon 0.2
db = DBSCAN(eps=0.2) 
bench_clustering2(fc=db, name="DBSCAN-f 0.2", data=MFCCs_data, labels=labels_data, all=False)

#DBSCAN con épsilon por defecto agregando outliers a cluster extra
db = DBSCAN() 
bench_clustering2(fc=db, name="DBSCAN-t 0.5", data=MFCCs_data, labels=labels_data, all=True)

#DBSCAN con épsilon 0.7 agregando outliers a cluster extra
db = DBSCAN(eps=0.7) 
bench_clustering2(fc=db, name="DBSCAN-t 0.7", data=MFCCs_data, labels=labels_data, all=True)

#DBSCAN con épsilon 0.2 agregando outliers a cluster extra
db = DBSCAN(eps=0.2) 
bench_clustering2(fc=db, name="DBSCAN-t 0.2", data=MFCCs_data, labels=labels_data, all=True)

#Clustering aglomerativo
Agg = AgglomerativeClustering(n_clusters=10) 
bench_clustering2(fc=Agg, name="Aglomerativo", data=MFCCs_data, labels=labels_data, all=False)

print(105 * '_')

_________________________________________________________________________________________________________
init			time	inertia	homo	compl	v-meas	ARI	AMI	silhouette	clusters
random         		0.694s	55364	0.666	0.517	0.582	0.329	0.581	0.237		10
k-means++      		0.510s	51637	0.695	0.584	0.635	0.552	0.634	0.245		10
DBSCAN-f 0.5   		0.575s	nan	1.000	0.327	0.493	0.256	0.483	0.235		8
DBSCAN-f 0.7   		0.714s	nan	1.000	0.585	0.738	0.450	0.730	0.509		13
DBSCAN-f 0.2   		0.375s	nan	1.000	0.000	0.000	0.000	0.000	0.517		2
DBSCAN-t 0.5   		0.580s	nan	0.074	0.451	0.127	0.049	0.123	-0.215		9
DBSCAN-t 0.7   		0.711s	nan	0.179	0.545	0.270	0.133	0.265	-0.163		14
DBSCAN-t 0.2   		0.388s	nan	0.011	0.677	0.022	0.006	0.021	-0.096		3
Aglomerativo   		2.035s	nan	0.789	0.661	0.719	0.590	0.718	0.276		10
_________________________________________________________________________________________________________


## Pruebas con PCA

In [ ]:
#Pruebas con PCA
print(105 * '_')
print('init\t\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette\tclusters')
#K-Means con inicialización al azar
reduced_data = PCA(n_components=2).fit_transform(MFCCs_data)
kmeans = KMeans(init='random', n_clusters=10)
bench_k_means(kmeans=kmeans, name="p-random", data=reduced_data, labels=labels_data)

#K-means++
kmeans = KMeans(init="k-means++", n_clusters=10)
bench_k_means(kmeans=kmeans, name="p-k-means++", data=reduced_data, labels=labels_data)

#DBSCAN con épsilon por defecto
db = DBSCAN() 
bench_clustering2(fc=db, name="p-DBSCAN-f 0.5", data=reduced_data, labels=labels_data, all=False)

#DBSCAN con épsilon 0.7
db = DBSCAN(eps=0.7) 
bench_clustering2(fc=db, name="p-DBSCAN-f 0.7", data=reduced_data, labels=labels_data, all=False)

#DBSCAN con épsilon 0.2
db = DBSCAN(eps=0.2) 
bench_clustering2(fc=db, name="p-DBSCAN-f 0.2", data=reduced_data, labels=labels_data, all=False)

#DBSCAN con épsilon por defecto agregando outliers a cluster extra
db = DBSCAN() 
bench_clustering2(fc=db, name="p-DBSCAN-t 0.5", data=reduced_data, labels=labels_data, all=True)

#DBSCAN con épsilon 0.7 agregando outliers a cluster extra
db = DBSCAN(eps=0.7) 
bench_clustering2(fc=db, name="p-DBSCAN-t 0.7", data=reduced_data, labels=labels_data, all=True)

#DBSCAN con épsilon 0.2 agregando outliers a cluster extra
db = DBSCAN(eps=0.2) 
bench_clustering2(fc=db, name="p-DBSCAN-t 0.2", data=reduced_data, labels=labels_data, all=True)

#Clustering aglomerativo
Agg = AgglomerativeClustering(n_clusters=10) 
bench_clustering2(fc=Agg, name="p-Aglomerativo", data=reduced_data, labels=labels_data, all=False)

print(105 * '_')

_________________________________________________________________________________________________________
init			time	inertia	homo	compl	v-meas	ARI	AMI	silhouette	clusters
p-random       		0.581s	944	0.654	0.496	0.564	0.348	0.563	0.395		10
p-k-means++    		0.307s	942	0.671	0.518	0.585	0.404	0.584	0.395		10
p-DBSCAN-f 0.5 		0.241s	nan	0.000	1.000	0.000	0.000	0.000	nan		1
p-DBSCAN-f 0.7 		0.250s	nan	-0.000	1.000	-0.000	0.000	-0.000	nan		1
p-DBSCAN-f 0.2          No se pueden calcular las métricas
p-DBSCAN-t 0.5          No se pueden calcular las métricas
p-DBSCAN-t 0.7          No se pueden calcular las métricas
p-DBSCAN-t 0.2 		0.109s	nan	0.004	0.103	0.007	0.001	0.005	0.354		4
p-Aglomerativo 		1.552s	nan	0.649	0.490	0.558	0.350	0.557	0.407		10
_________________________________________________________________________________________________________
